In [12]:
import glob
import os
from pathlib import Path
from random import Random

lista = glob.glob("<path>/Nowy folder/notsmoking*")
for file in lista:
    Path(file).rename(f"<path>/not_smoking/{os.path.basename(file)}")

lista = glob.glob("<path>/Nowy folder/smoking*")
for file in lista:
    Path(file).rename(f"<path>/smoking/{os.path.basename(file)}")

In [23]:
paths = [
    ".../Szum/not_smoking",
    ".../Szum/smoking",
    ".../Szum/Other"
]

In [ ]:
from PIL import Image

for path in paths:
    print(path)
    small_photo = []
    for filename in os.listdir(path):
        filepath = os.path.join(path, filename)

        try:
            with Image.open(filepath) as img:
                width, height = img.size
                if width < 200 or height < 200:
                    small_photo.append(filepath)
        except Exception as e:
            print(f"Nie udało się otworzyć {filename}: {e}")

    try:
        os.mkdir(f"{path}\\small_photo")
    except FileExistsError:
        pass
    for file in small_photo:
        Path(file).rename(f"{path}/small_photo/{os.path.basename(file)}")

In [34]:
for path in paths:
    try:
        os.mkdir(f"{path}/not_jpg")
    except FileExistsError:
        pass
    for filename in [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]:
        if Path(filename).suffix.lower() not in ['.jpg', '.jpeg']:
            Path(f"{path}/{filename}").rename(f"{path}/not_jpg/{filename}")

In [ ]:
for path in paths:
    print("Number of images in:", path, "=", len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]))

In [ ]:
import random as rnd

random = rnd.Random()
random.seed(6042025)
selected_images = []
for path in paths:
    filename_list = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    random.shuffle(filename_list)
    selected_images.append(filename_list[:1500])

